In [ ]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
data = pd.concat(( train, test ))
train = []
test = []

In [4]:
#remember to delete this
print("msno")
msno_values = data.msno.unique()
print("end msno")

msno
end msno


In [5]:
#user_logs = pd.read_csv('cleaned_input/user_logs_merged.csv')
chunksize = 1000000
user_logs = None
int i = 0 
for visit in pd.read_csv('raw_data/user_logs.csv', chunksize=chunksize, index_col=['msno'], usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    print(i)
    if user_logs is None:
        user_logs = visit.copy()
    else:
        user_logs = pd.concat((user_logs, visit))
    i += 1 
user_logs.head()

                                                  date  num_unq  total_secs
msno                                                                       
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150513        1     280.335
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150709       11    1658.948
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150105       36   17364.956
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150306       27   24667.317
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150501       38    9649.029
                                                  date  num_unq  total_secs
msno                                                                       
Dg3WlnWHxkJ5EeVuce7yyEY0AbKANn7kLimi4PiDhns=  20160506       32    5164.286
Dg3WlnWHxkJ5EeVuce7yyEY0AbKANn7kLimi4PiDhns=  20160910       23    4933.167
Dg3WlnWHxkJ5EeVuce7yyEY0AbKANn7kLimi4PiDhns=  20161226       21     525.772
yJfiHA29Qeiq01sP80U5RLZLiMh28Jsy4SISZ+H1jeM=  20161202       35    9000.233
uwv4Hj7YmV9S

KeyboardInterrupt: 

In [ ]:
user_logs_2 = pd.read_csv('raw_data/user_logs_v2.csv', index_col=['msno'], usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float})
user_logs = pd.concat((user_logs, user_logs_2))

In [ ]:
user_logs.to_csv("cleaned_input/user_logs_merged_slimmer.csv")

In [ ]:
print("msno")
msno_values = data.msno.unique()
print("end msno")

In [ ]:
print("clean up")
user_logs_cleaned = user_logs[user_logs['msno'].isin(msno_values)]

In [ ]:
user_logs_cleaned.to_csv('cleaned_input/user_logs_cleaned.csv')

In [ ]:
print(user_logs_2.shape)
print(user_logs.shape)
print(user_logs_cleaned.shape)
user_logs.head()

In [ ]:
#get number of unique visits
user_features = pd.DataFrame() 
counts = user_logs.msno.value_counts()

user_features['msno'] = counts.index
user_features["frequency"] = counts.values

In [ ]:
'''date_cols = ['date']

for col in date_cols:
    user_logs[col] = pd.to_datetime(user_logs[col], format='%Y%m%d')'''
user_logs['date'] = pd.to_datetime(user_logs['date'], format='%Y%m%d')

In [ ]:
#sort user_logs by msno and then date
user_logs = user_logs.sort_values(['msno', 'date'], ascending=[True, True])

In [ ]:
user_logs.head()

In [ ]:
rows = user_logs.msno.unique()

In [ ]:
print("start")
user_features["gap_days"] = np.nan
user_features["average_gap_days"] = np.nan
user_features["longest_gap"] = np.nan
user_features["gaps_longer_29"] = np.nan
user_features.head()

In [ ]:
#find the differences between days and average difference 
for row in rows:
    print(row)
    df = user_logs[user_logs["msno"] == row]
    differences = df.date.diff()
    print(differences)
    gap_days = differences.sum()
    average_gap_days = differences.mean()
    longest_gap = differences.nlargest(1)
    gaps_longer_29 = 0
    for val in differences:
        if val > pd.Timedelta('29 days'):
            gaps_longer_29 += 1
    user_features[user_features['msno'] == row]["gap_days"] = gap_days
    user_features[user_features['msno'] == row]["average_gap_days"] = average_gap_days
    user_features[user_features['msno'] == row]["longest_gap"] = longest_gap
    user_features[user_features['msno'] == row]["gaps_longer_29"] = gaps_longer_29
    

In [ ]:
user_features.head()

In [ ]:
user_features.to_csv("userlogs_featueres.csv")